# Extract Earnings Releases

This notebook shows how to get the earnings release from an **8-K** filing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/Extract-Earnings-Releases.ipynb)

In [ ]:
!pip install edgartools

In [1]:
from edgar import *
from edgar.files.html import HtmlDocument

set_identity("me@myemail.com")

## Find a Filing

In [2]:
filing = Filing(company='Apple Inc.', cik=320193, form='8-K', filing_date='2024-08-01', accession_no='0000320193-24-000080')

## Attachments

Get the filing attachments

In [3]:
attachments = filing.attachments
attachments

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                         Documents                                                               │
│                                                                                                                 │
│   Seq   Document                      Description                     Type      Size                            │
│  ────────────────────────────────────────────────────────────────────────────────────────                       │
│   1     aapl-20240801.htm             8-K                             8-K       40.2 KB                         │
│   2     a8-kex991q3202406292024.htm   EX-99.1                         EX-99.1   164.5 KB                        │
│   7     aapl-20240801_g1.jpg                                          GRAPHIC   1.2 KB                          │
│         0000320193-24-000080.txt      Complete submission text file   

### Get the attachments that are exhibits
Some filings are actually exhibits. Call `exhibits` to get just the exhibits.

In [4]:
exhibits = attachments.exhibits
exhibits

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                Documents                                                                        │
│                                                                                                                 │
│   Seq   Document                      Description   Type      Size                                              │
│  ──────────────────────────────────────────────────────────────────────                                         │
│   1     aapl-20240801.htm             8-K           8-K       40.2 KB                                           │
│   2     a8-kex991q3202406292024.htm   EX-99.1       EX-99.1   164.5 KB                                          │
│                                                                                                                 │
╰───────────────────────────────────────────────────────────────────────

## Select an exhibit by index

In [10]:
exhibits[1]

╭─────────────────────────────┬─────────────┬─────────┬──────────╮
│ Document                    │ Description │ Type    │ Size     │
├─────────────────────────────┼─────────────┼─────────┼──────────┤
│ a8-kex991q3202406292024.htm │ EX-99.1     │ EX-99.1 │ 164.5 KB │
╰─────────────────────────────┴─────────────┴─────────┴──────────╯

## Query for just exhibit EX-99.1
Alternatively, you can use the query function to make sure you are selecting only EX-99.1. Not all filings have this exhibit

```python
results = exhibits.query("document_type in ['EX-99.1', 'EX-99', 'EX-99.01']")
if len(results) > 0:
    print(results[0])
```

## Download the html content of the exhibit

In [6]:
html = exhibits[1].download()

## Extract the text of the exhibit

In [7]:
exhibit_text = HtmlDocument.from_html(exhibits[1].download()).text
print(exhibit_text)

Document

Exhibit 99.1
Apple reports third quarter results
June quarter records for Revenue and EPS
Services revenue reaches new all-time high
CUPERTINO, CALIFORNIA — Apple® today announced financial results for its fiscal 2024 third quarter ended June 29, 2024. The Company posted quarterly revenue of $85.8 billion, up 5 percent year over year, and quarterly earnings per diluted share of $1.40, up 11 percent year over year.
“Today Apple is reporting a new June quarter revenue record of $85.8 billion, up 5 percent from a year ago,” said Tim Cook, Apple’s CEO. “During the quarter, we were excited to announce incredible updates to our software platforms at our Worldwide Developers Conference, including Apple Intelligence, a breakthrough personal intelligence system that puts powerful, private generative AI models at the core of iPhone, iPad, and Mac. We very much look forward to sharing these tools with our users, and we continue to invest significantly in the innovations that will enrich

In [9]:
exhibits.query("document_type in ['EX-99.1', 'EX-99', 'EX-99.01']")

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                Documents                                                                        │
│                                                                                                                 │
│   Seq   Document                      Description   Type      Size                                              │
│  ──────────────────────────────────────────────────────────────────────                                         │
│   2     a8-kex991q3202406292024.htm   EX-99.1       EX-99.1   164.5 KB                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯